#--- Social Media-Ready Editing ---

In [ ]:
def apply_social_media_edits(pil_image, preferred_aspect_ratio=(4,5)):
    """Applies common social media edits: crop, enhance, sharpen."""
    img_width, img_height = pil_image.size

    # Smart Cropping (AI for object-aware cropping is a future enhancement)
    target_w, target_h = preferred_aspect_ratio

    potential_h = int(img_width * (target_h / target_w))
    if potential_h <= img_height:
        crop_width = img_width
        crop_height = potential_h
        left = 0
        top = (img_height - crop_height) // 2
    else:
        crop_height = img_height
        crop_width = int(img_height * (target_w / target_h))
        left = (img_width - crop_width) // 2
        top = 0

    right = left + crop_width
    bottom = top + crop_height

    pil_image = pil_image.crop((left, top, right, bottom))

    # Basic Enhancements (can be AI-stylized in future)
    enhancer = ImageEnhance.Contrast(pil_image)
    pil_image = enhancer.enhance(1.1)

    enhancer = ImageEnhance.Color(pil_image)
    pil_image = enhancer.enhance(1.1)

    pil_image = pil_image.filter(ImageFilter.SHARPEN)

    return pil_image

# --- Album-Level Vibe Caption Generation (Fully AI-powered) ---

In [ ]:
class AlbumVibeCaptionGenerator:
    def __init__(self):
        print("\nLoading AI models for captioning and vibe generation...")
        # BLIP for individual image descriptions (Vision-Encoder-Decoder)
        self.blip_captioner = pipeline("image-to-text", model=BLIP_MODEL_NAME, device=0 if torch.cuda.is_available() else -1)
        print(f"Loaded BLIP model: {BLIP_MODEL_NAME}")

        # Gemma for LLM-based album caption summarization and vibe generation
        # Ensure you have accepted the model terms on Hugging Face and set up HF_TOKEN secret in Colab
        try:
            self.llm_tokenizer = AutoTokenizer.from_pretrained(GEMMA_LLM_MODEL)
            self.llm_model = AutoModelForCausalLM.from_pretrained(
                GEMMA_LLM_MODEL,
                torch_dtype=torch.float16, # Use float16 for memory efficiency
                device_map="auto", # Automatically map to GPU if available
                token=os.environ.get("HF_TOKEN") # Use HF_TOKEN from Colab Secrets
            )
            print(f"Loaded LLM model: {GEMMA_LLM_MODEL}")
        except Exception as e:
            print(f"ERROR: Could not load LLM model '{GEMMA_LLM_MODEL}'.")
            print("Please ensure you have accepted its terms on Hugging Face and set up your HF_TOKEN secret in Colab.")
            print(f"Details: {e}")
            self.llm_tokenizer = None
            self.llm_model = None

    def generate_individual_captions(self, pil_images):
        """Generates captions for each image using the BLIP model."""
        individual_captions = []
        if self.blip_captioner:
            print("Generating individual image descriptions...")
            for i, img in enumerate(pil_images):
                try:
                    results = self.blip_captioner(img)
                    if results:
                        caption = results[0]['generated_text']
                        individual_captions.append(f"Image {i+1}: {caption}")
                except Exception as e:
                    print(f"  Error captioning image {i+1}: {e}")
                    individual_captions.append(f"Image {i+1}: Could not describe this image.")
        return individual_captions

    def generate_album_vibe_caption(self, individual_captions):
        """
        Generates a single, vibe-fitting caption for the entire album using an LLM.
        """
        if not self.llm_model:
            return "Could not generate album caption due to LLM loading error. Check Colab setup."

        # Craft a prompt for the LLM based on the individual captions
        context = "\n".join(individual_captions)

        prompt = textwrap.dedent(f"""
        You are a highly creative and engaging social media caption generator for influencers.
        You have analyzed a collection of photos from an album.
        Below are individual descriptions of the key photos in the album.

        Based on these descriptions, identify the overall theme, mood, and "vibe" of the entire album.
        Then, generate a single, compelling, and short social media caption (max 3 sentences) that captures this vibe.
        Include relevant emojis and 3-5 popular hashtags to maximize engagement.
        Make it sound authentic and inspiring for social media influencers.

        Individual photo descriptions:
        ---
        {context}
        ---

        Album Vibe Caption:
        """)

        # Tokenize and generate with the LLM

        inputs = self.llm_tokenizer(prompt, return_tensors="pt").to(self.llm_model.device)

        outputs = self.llm_model.generate(
            **inputs,
            max_new_tokens=100, # Adjust for desired length of the album caption
            temperature=0.8,
            do_sample=True,
            pad_token_id=self.llm_tokenizer.eos_token_id
        )

        generated_text = self.llm_tokenizer.decode(outputs[0], skip_special_tokens=True)

        response_start = generated_text.find("Album Vibe Caption:")
        if response_start != -1:
            generated_caption = generated_text[response_start + len("Album Vibe Caption:"):].strip()
        else:
            generated_caption = generated_text

        generated_caption = generated_caption.split("Individual photo descriptions:")[0].strip()

        if not any(tag.startswith('#') for tag in generated_caption.split()):
            generated_caption += "\n#AlbumVibes #InfluencerLife #ContentCreator"

        return generated_caption

# --- Main Processing Logic ---

In [ ]:
def process_album(album_path):
    if not os.path.exists(album_path):
        print(f"Error: Album path '{album_path}' does not exist.")
        return

    os.makedirs(OUTPUT_DIR, exist_ok=True)

    # Initialize AI models
    album_caption_generator = AlbumVibeCaptionGenerator() # Handles both individual and album-level AI captioning
    aesthetic_scorer = AestheticScorer() # AI aesthetic scoring (now a real model)

    all_photos_info = [] # To store loaded images and their scores

    # 1. Gather all photos and process Live Photos (AI-informed best shot)
    photo_paths = get_image_paths(album_path)

    if not photo_paths:
        print(f"No image files found in '{album_path}'. Please check the directory and file types.")
        return

    for i, photo_entry in enumerate(photo_paths):
        print(f"[{i+1}/{len(photo_paths)}] Processing {photo_entry['type']} photo...")
        pil_image = None
        original_path = ""

        if photo_entry['type'] == 'live':
            original_path = photo_entry['jpg_path']
            best_frame = select_best_live_photo_frame(photo_entry['mov_path'])
            if best_frame:
                pil_image = best_frame
                print(f"Selected best frame from Live Photo: {os.path.basename(original_path)}")
            else:
                print(f"Could not process Live Photo: {os.path.basename(original_path)}. Skipping.")
                continue
        else: # Still photo
            original_path = photo_entry['path']
            try:
                pil_image = Image.open(original_path).convert('RGB')
                print(f"Successfully loaded still photo: {os.path.basename(original_path)}")
            except Exception as e:
                print(f"Error opening still photo {original_path}: {e}. Skipping.")
                continue

        if pil_image:
            # Here's where the *real* AI aesthetic score is used
            score = score_photo_engagement(pil_image, aesthetic_scorer)
            all_photos_info.append({
                'original_path': original_path,
                'pil_image': pil_image,
                'score': score
            })
            print(f"  -> Scored {os.path.basename(original_path)}: {score:.2f} (AI Aesthetic Score)")

    # 2. Select top N photos based on AI-driven score
    all_photos_info.sort(key=lambda x: x['score'], reverse=True)
    selected_photos_info = all_photos_info[:MAX_PHOTOS_TO_SELECT]

    if not selected_photos_info:
        print("No suitable photos found or selected based on AI aesthetic and engagement scoring. Try adding more photos or adjusting MAX_PHOTOS_TO_SELECT.")
        return

    print(f"\nSelected {len(selected_photos_info)} best photos based on AI aesthetic and engagement scoring.")

    # Store individual processed images for album captioning
    processed_pil_images_for_captioning = []

    # 3. Process selected photos: edit, save
    print("\nApplying social media edits and saving selected photos...")
    for idx, photo_data in enumerate(selected_photos_info):
        original_img_path = photo_data['original_path']
        edited_pil_image = photo_data['pil_image'] # Start with original or best frame

        # Apply edits (you can customize the aspect ratio here)
        edited_pil_image = apply_social_media_edits(edited_pil_image, preferred_aspect_ratio=TARGET_ASPECT_RATIOS[0])

        # Save edited image
        output_filename = f"post_{idx+1}_{os.path.basename(original_img_path)}"
        output_filepath = os.path.join(OUTPUT_DIR, output_filename)
        edited_pil_image.save(output_filepath)
        processed_pil_images_for_captioning.append(edited_pil_image) # Add to list for album captioning

        print(f"  Saved edited photo: {output_filepath}")

    # 4. Generate a single, vibe-fitting caption for the entire album (Fully AI-powered)
    print("\nGenerating album-level caption using LLM...")
    individual_blip_captions = album_caption_generator.generate_individual_captions(processed_pil_images_for_captioning)
    album_vibe_caption = album_caption_generator.generate_album_vibe_caption(individual_blip_captions)

    # Write the album caption to a file
    album_caption_filepath = os.path.join(OUTPUT_DIR, "album_vibe_caption.txt")
    with open(album_caption_filepath, 'w', encoding='utf-8') as f_album_caption:
        f_album_caption.write("--- Album Vibe Caption for All Selected Photos ---\n\n")
        f_album_caption.write(textwrap.fill(album_vibe_caption, width=80)) # Wrap for readability
        f_album_caption.write("\n\n--- Individual Photo Descriptions (for reference) ---\n\n")
        for cap in individual_blip_captions:
            f_album_caption.write(textwrap.fill(cap, width=80) + "\n")

    print(f"\nAll processed photos saved to: {os.path.abspath(OUTPUT_DIR)}")
    print(f"Album vibe caption and individual descriptions saved to: {os.path.abspath(album_caption_filepath)}")
    print("\nProcessing Complete!")